<a href="https://colab.research.google.com/github/aheiX/Teaching/blob/main/Aggregierte_Produktionsplanung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aggregierte Produktionsplanung

## Aufgabenstellung

**Entscheidungsvariablen**<br>
$
\begin{array}{ll}
H_t & \text{Anzahl der Einstellungen am Anfang von Periode } t\\
F_t & \text{Anzahl der Entlassungen am Anfang von Periode } t\\
W_t & \text{Mitarbeiterkapazität: Anzahl der Mitarbeiter am Anfang von Periode } t\\
I_t & \text{Lagerbestand am Anfang von Periode } t\\
X_t & \text{Produktionsmenge in Periode } t\\
S_t & \text{Fremdvergabe in Stück in Periode } t\\
O_t & \text{Überstunden Mitarbeiter-Äquivalente in Periode } t\\
\end{array}
$
<br><br>
**Parameter**<br>
$
\begin{array}{ll}
c_H & \text{Einstellungskostensatz 10.000 Euro pro Mitarbeiter}\\
c_F & \text{Entlassungskostensatz 5.000 Euro pro Mitarbeiter}\\
c_W & \text{Personalkostensatz 6.000 Euro pro Mitarbeiter}\\
c_I & \text{Lagerhaltungskostensatz 40 Euro pro Stück}\\
c_S & \text{Fremdvergabekostensatz 790 Euro pro Stück}\\
c_O & \text{Überstundenkosten 10000 Euro pro Mitarbeiter}\\
Y_t & \text{Nachfrage in Periode } t \text{ (2.500, 2.000, 3.160, 1.500)}\\
PK & \text{Produktionskoeffizient 0,069444 Mitarbeiter pro Stück}\\
T & \text{Planungshorizont 4 Monate}\\
\end{array}
$

## Mathematisches Modell

**Zielfunktion**<br><br>
$\min \sum\limits_{t=1}^{T} c_H \cdot H_t + c_F \cdot F_t + c_w \cdot W_t + c_I \cdot I_{t+1}$
<br><br>

**Nebenbedingungen**<br>
$
\begin{array}{rclll}
W_t & = & W_{t-1} + H_t -F_t &, \forall t=1,...,T& \hspace{1.5cm}(1)\\
X_t & \le & \frac{1}{PK} \cdot W_{t} &, \forall t=1,...,T& \hspace{1.5cm}(2)\\
I_{t+1} & = & I_t + X_t - Y_t &, \forall t=1,...,T& \hspace{1.5cm}(3)\\
I_1 & = & 0 & & \hspace{1.5cm}(4)\\
W_0 & = & 0 & & \hspace{1.5cm}(5)\\
H_t, F_t, W_t, X_t, I_{t+1} & \ge & 0 &, \forall t=1,...,T& \hspace{1.5cm}(6)\\
\end{array}
$



## Implementierung

### Daten aufbereiten

In [7]:
# Daten als Dictionary
c_H = 10000
c_F = 5000
c_W = 6000
c_I = 40
c_S = 790   # Fremdvergabe
c_O = 10000 # Überstunden
Y = {1: 2500, 2: 2000, 3: 3160, 4: 1500}
PK = 0.069444
T = [1, 2, 3, 4]

### PuLP Model

In [3]:
# PuLP installieren
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 77.5 MB/s eta 0:00:00


In [14]:
# Modell erstellen
model = pulp.LpProblem(name='Aggregierte_Produktionsplanung', sense=pulp.constants.LpMinimize)

# Entscheidungsvariablen
H = pulp.LpVariable.dicts(name='H', indices=T, lowBound=0)
F = pulp.LpVariable.dicts(name='F', indices=T, lowBound=0)
W = pulp.LpVariable.dicts(name='W', indices=[0] + T, lowBound=0)
I = pulp.LpVariable.dicts(name='I', indices=T + [5], lowBound=0)
X = pulp.LpVariable.dicts(name='X', indices=T, lowBound=0)
S = pulp.LpVariable.dicts(name='S', indices=T, lowBound=0) # Fremdvergabe
O = pulp.LpVariable.dicts(name='O', indices=T, lowBound=0) # Überstunden

# Zielfunktion
model += sum(c_H * H[t] + c_F * F[t] + c_W * W[t] + c_I * I[t+1]
            #  + c_S * S[t] # Fremdvergabe
            #  + c_O + O[t] # Überstunden
             for t in T)

# Arbeitskräfte
for t in T:
  model += W[t] == W[t-1] + H[t] - F[t]

# Kapazität
for t in T:
  model += X[t] <= (1/PK) * W[t]
  # model += X[t] <= (1/PK) * (W[t] + O[t]) # Überstunden
  # model += O[t] <= 20                     # Überstunden

# Lagerbestand
for t in T:
  model += I[t+1] == I[t] + X[t] - Y[t]
  # model += I[t+1] == I[t] + X[t] + S[t] - Y[t] # Fremdvergabe

# Anfangsbedingungen
model += I[1] == 0
model += W[0] == 100

# Bedarfsverfolgung
# for t in T:
#   model += W[t] == PK * Y[t]

# Kapazitätsfixierung
# for t in T[1:]:
#   model += W[t] == W[t-1]

# print(model)

# Modell lösen
model.solve()

# Statistik
print('Status:', pulp.LpStatus[model.status])
print('Zielwert:', pulp.value(model.objective))
for v in model.variables():
  if v.varValue > 0:
    print(v.name, '=', v.varValue)

Status: Optimal
Zielwert: 4981917.44012
F_4 = 73.14768
H_1 = 77.31368
I_2 = 53.333333
I_3 = 606.66667
W_0 = 100.0
W_1 = 177.31368
W_2 = 177.31368
W_3 = 177.31368
W_4 = 104.166
X_1 = 2553.3333
X_2 = 2553.3333
X_3 = 2553.3333
X_4 = 1500.0
